In [74]:
# !pip install pandas

In [75]:
# Libraries
import pandas as pd
input_folder_path = "DataSource/Input/"
output_folder_path = "DataSource/Output/"

#### Game Title Process Method

In [76]:
def title_process (df):
    # remove everything in front of last column
    df['Game Title'] = df['Game Title'].str.replace(' : ', ':', regex = False).str.rsplit(':', n=1).str[-1].str.upper()

    # remove platform
    # pattern = '|'.join(platforms)
    # df['Game Title'] = df['Game Title'].str.replace(rf'\s*\b({pattern})\b.*', '', regex=True)
    
    return df

#### Item Code Process Method

In [77]:
def item_process (df):
    df['Item'] = df['Item'].apply(lambda x: '-'.join(x.split('-')[:2]))
    return df

### Sales Data Process

In [78]:
file_name = "Sales.xlsx"
df = pd.read_excel(input_folder_path + file_name, skiprows=4)

# Reset the index
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df = df.reset_index(drop=True)

#cleaning up
df = df[['Game Title', 'Item', 'Platform', 'Sales (USD)']]
platforms = df['Platform'].unique().tolist()
platforms = [str(platform) for platform in platforms]
platforms.append('STEAM')
df.dropna(subset = ['Item'], inplace = True)

# group by item
df = title_process(df)
df = item_process(df)
df = df.groupby('Item').agg({
    'Game Title': 'first',
    'Sales (USD)': 'sum'
}).reset_index()

display(df)

,Item,Game Title,Sales (USD)
0,AEC1-PSP,AEDIS ECLIPSE-PSN,10.47
1,AHL3-PS3,ARCANA HEART 3 PAL PS3,78.5
2,AHL3-PSV,ARCANA HEART 3 PAL PSV,9.31
3,AKB2-PS3,AKIBA TRIP 2,44.49
4,AKB2-PS4,AKIBA TRIP 2,690.13
...,...,...,...
368,YUK1-NSW,YURUKILL NSW,732.12
369,YUK1-PS4,YURUKILL PS4,72.27
370,YUK1-PS5,YURUKILL PS5,720.12
371,ZGEN-GDS,- NO GAME TITLE -,841.12


### COGS data process

In [79]:
# Data Cleanup
COGS_df = pd.read_csv(input_folder_path + "COGS.csv", skiprows=4)
COGS_df.columns = COGS_df.iloc[0]
COGS_df = COGS_df.drop(COGS_df.index[0])
COGS_df = COGS_df.reset_index(drop=True)
COGS_df = COGS_df.dropna(subset = ['Game Title: Name'])


# Grouping by item
COGS_df['Amount'] = COGS_df['Amount'].str.replace('$', '').str.replace(',', '').str.strip().astype(float)
COGS_df = COGS_df.rename(columns = {'Amount': 'COGS Expense'})
COGS_df = COGS_df.rename(columns = {'Game Title: Name': 'Game Title'})
COGS_df = title_process(COGS_df)
COGS_df = item_process(COGS_df)
COGS_df = COGS_df.groupby('Item')['COGS Expense'].sum().reset_index()

display(COGS_df)

,Item,COGS Expense
0,- No Item,11026.79
1,CCS1-BRD,285.66
2,D6C1-PS4,160.45
3,D6C1-PS5,187.54
4,DIS7-NSW,2670.77
...,...,...
62,YSN1-OTH,453.09
63,YSN1-PS4,39691.14
64,YSN1-PS5,239648.19
65,YUK1-NSW,50.82


### Capitalized Expense Process

In [80]:
# clean up
CapExp_df = pd.read_excel(input_folder_path + "Capitalized_Expense.xlsx")
CapExp_df['Capitalized Expense Amount (Debit + Credit)'] = CapExp_df['Amount (Debit)'] + CapExp_df['Amount (Credit)']
CapExp_df.dropna(subset = ['Game Title'], inplace = True)

# grouping by title
CapExp_df = title_process(CapExp_df)
CapExp_df = CapExp_df.groupby('Game Title')['Capitalized Expense Amount (Debit + Credit)'].sum().reset_index()
display(CapExp_df)


,Game Title,Capitalized Expense Amount (Debit + Credit)
0,REYNATIS,35608.15
1,TRAILS THROUGH DAYBREAK,240404.69
2,TRAILS THROUGH DAYBREAK STEAM,81557.49


### AD Promotion Data Process

In [81]:
# clean up
AD_df = pd.read_excel(input_folder_path + "AD_Promotion.xlsx", skiprows=4)
AD_df.columns = AD_df.iloc[0]
AD_df = AD_df.drop(AD_df.index[0])
AD_df = AD_df.reset_index(drop=True)
AD_df = AD_df.dropna(subset = ['Game Title: Name'])

# grouping by title
AD_df = AD_df.rename(columns = {'Amount': 'AD Promotion'})
AD_df = AD_df.rename(columns = {'Game Title: Name': 'Game Title'})
AD_df = title_process(AD_df)
AD_df = AD_df.groupby('Game Title')['AD Promotion'].sum().reset_index()


display(AD_df)

,Game Title,AD Promotion
0,NORDICS,2960
1,- NO GAME TITLE -,4495.77
2,2024-07_ANIME EXPO 2024,386.02
3,2024-08_PAX WEST,53860.14
4,PHANTOM BRAVE 1 PS5,1058.97
5,PHANTOM BRAVE 2,9.25


### Royalty Expense Process

In [82]:
# get data
Royalty_df = pd.read_excel(input_folder_path + "Royalty.xlsx", sheet_name='Royalty Billing Record')
Royalty_df.rename(columns = {'game_title': 'Game Title'}, inplace = True)
Royalty_df.rename(columns = {'item': 'Item'}, inplace = True)
Royalty_df.dropna(subset = ['Game Title'], inplace = True)


# grouping by title
Royalty_df = title_process(Royalty_df)
Royalty_df = item_process(Royalty_df)
Royalty_df = Royalty_df.groupby('Item')['sales'].sum().reset_index()
Royalty_df.rename(columns = {'sales': 'Royalty Expense'}, inplace = True)

display(Royalty_df)

,Item,Royalty Expense
0,AHL3-PS3,133.66
1,AKB2-PS3,48.06
2,AKB2-PS4,581.14
3,AKB2-PSV,25.00
4,ASP1-STM,2228.88
...,...,...
230,YSM1-STM,25920.48
231,YSM1-TOM,3799.65
232,YSN1-OTH,1441.23
233,YUK1-NSW,1748.78


### Merging  and Export Data

In [83]:
# Merging
df = pd.merge(df, COGS_df, how='left', on='Item')
df = pd.merge(df, Royalty_df, how='left', on='Item')
df = pd.merge(df, CapExp_df, how='left', on='Game Title')
df = pd.merge(df, AD_df, how='left', on='Game Title')

# Calculate profit
df['Sales (USD)'] = df['Sales (USD)'].astype(float)
df['Profit'] = df['Sales (USD)'].fillna(0) - df['COGS Expense'].fillna(0) - df['Royalty Expense'].fillna(0) - df['Capitalized Expense Amount (Debit + Credit)'].fillna(0) - df['AD Promotion'].fillna(0)

display(df)

C:\Users\dentsusokendx_user2\AppData\Local\Temp\ipykernel_9296\2555966255.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Profit'] = df['Sales (USD)'].fillna(0) - df['COGS Expense'].fillna(0) - df['Royalty Expense'].fillna(0) - df['Capitalized Expense Amount (Debit + Credit)'].fillna(0) - df['AD Promotion'].fillna(0)


,Item,Game Title,Sales (USD),COGS Expense,Royalty Expense,Capitalized Expense Amount (Debit + Credit),AD Promotion,Profit
0,AEC1-PSP,AEDIS ECLIPSE-PSN,10.47,NaN,NaN,NaN,NaN,10.47
1,AHL3-PS3,ARCANA HEART 3 PAL PS3,78.50,NaN,133.66,NaN,NaN,-55.16
2,AHL3-PSV,ARCANA HEART 3 PAL PSV,9.31,NaN,NaN,NaN,NaN,9.31
3,AKB2-PS3,AKIBA TRIP 2,44.49,NaN,48.06,NaN,NaN,-3.57
4,AKB2-PS4,AKIBA TRIP 2,690.13,NaN,581.14,NaN,NaN,108.99
...,...,...,...,...,...,...,...,...
368,YUK1-NSW,YURUKILL NSW,732.12,50.82,1748.78,NaN,NaN,-1067.48
369,YUK1-PS4,YURUKILL PS4,72.27,NaN,NaN,NaN,NaN,72.27
370,YUK1-PS5,YURUKILL PS5,720.12,NaN,388.44,NaN,NaN,331.68
371,ZGEN-GDS,- NO GAME TITLE -,841.12,22400.39,NaN,NaN,4495.77,-26055.04


In [84]:
df.to_csv(output_folder_path + 'Sales and Profit Data.csv', index = True)